In [ ]:
import pandas as pd
import numpy as np
import datetime as dt

In [ ]:
data = pd.read_csv("./all_aq_data.csv")
data.drop(columns='Unnamed: 0', inplace=True)

In [ ]:
data.groupby(['Site.ID', 'school_year']).ngroups

2950

In [ ]:
# Convert 'Date' column to datetime format
data['Date'] = pd.to_datetime(data['Date'], format='%Y/%m/%d')

# Sort the DataFrame by 'Site.ID' and 'Date'
data.sort_values(['Site.ID', 'Date'], inplace=True)

In [ ]:
valid_sites = data

In [ ]:
valid_sites.groupby(['Site.ID', 'school_year']).ngroups

2950

In [ ]:
valid_sites[valid_sites['school_year'] == 2002]

,Date,Site.ID,Daily.Mean.PM2.5.Concentration,SITE_LATITUDE,SITE_LONGITUDE,year,month,day,school_year,aq_vuh,count
116655,2002-03-03,60010007,13.3,37.687526,-121.784217,2002,3,3,2002,0,96
122145,2002-03-06,60010007,2.8,37.687526,-121.784217,2002,3,6,2002,0,96
127605,2002-03-09,60010007,8.1,37.687526,-121.784217,2002,3,9,2002,0,96
133092,2002-03-12,60010007,7.3,37.687526,-121.784217,2002,3,12,2002,0,96
138533,2002-03-15,60010007,5.4,37.687526,-121.784217,2002,3,15,2002,0,96
...,...,...,...,...,...,...,...,...,...,...,...
83874,2003-02-14,61131003,12.0,38.661210,-121.732690,2003,2,14,2002,0,114
89645,2003-02-17,61131003,5.0,38.661210,-121.732690,2003,2,17,2002,0,114
95414,2003-02-20,61131003,2.0,38.661210,-121.732690,2003,2,20,2002,0,114
101146,2003-02-23,61131003,5.0,38.661210,-121.732690,2003,2,23,2002,0,114


In [ ]:
def interpolate_monthly(group):
    # Ensure the group is sorted by Date
    group = group.sort_values('Date')

    # Get the year and month from the first row of the group
    year_month = group['Date'].dt.to_period('M').iloc[0]
    start_of_month = year_month.start_time
    end_of_month = year_month.end_time

    # Create a date range for the full month
    full_month_range = pd.date_range(start=start_of_month, end=end_of_month)

    # Set the index to the date for resampling purposes
    group = group.set_index('Date')

    # Reindex the group with the full month date range, introducing NaNs for missing dates
    group = group.reindex(full_month_range)

    # Fill the 'year' and 'month' columns
    group['year'] = group.index.year
    group['month'] = group.index.month

    # Backward fill the site information columns where we have NaNs due to reindexing
    group[['Site.ID', 'SITE_LATITUDE', 'SITE_LONGITUDE', 'school_year']] = group[['Site.ID', 'SITE_LATITUDE', 'SITE_LONGITUDE', 'school_year']].bfill().ffill()

    # Calculate the maximum gap of consecutive missing days
    # This will include gaps at the beginning or the end of the month
    group['consecutive_missing'] = group['Daily.Mean.PM2.5.Concentration'].isna().astype(int).groupby(group['Daily.Mean.PM2.5.Concentration'].notna().astype(int).cumsum()).cumsum()
    max_gap = group['consecutive_missing'].max()

    # Interpolate the missing values for 'Daily.Mean.PM2.5.Concentration'
    group['Daily.Mean.PM2.5.Concentration'] = group['Daily.Mean.PM2.5.Concentration'].interpolate(method='linear')

    # After interpolation, assign the max gap to the entire group
    group['max_consecutive_missing'] = max_gap

    # Reset the index so 'Date' becomes a column again
    return group.reset_index().rename(columns={'index': 'Date'})

In [ ]:
interpolated_data = valid_sites.groupby(['Site.ID', 'month', 'year']).apply(interpolate_monthly).reset_index(drop=True)

In [ ]:
interpolated_data[interpolated_data["school_year"] == 2003]

,Date,Site.ID,Daily.Mean.PM2.5.Concentration,SITE_LATITUDE,SITE_LONGITUDE,year,month,day,school_year,aq_vuh,count,consecutive_missing,max_consecutive_missing
31,2004-01-01,60010007.0,11.0,37.687526,-121.784217,2004,1,1.0,2003.0,0.0,135.0,0,1
32,2004-01-02,60010007.0,12.2,37.687526,-121.784217,2004,1,2.0,2003.0,0.0,135.0,0,1
33,2004-01-03,60010007.0,16.5,37.687526,-121.784217,2004,1,3.0,2003.0,0.0,135.0,0,1
34,2004-01-04,60010007.0,19.5,37.687526,-121.784217,2004,1,4.0,2003.0,0.0,135.0,0,1
35,2004-01-05,60010007.0,11.5,37.687526,-121.784217,2004,1,5.0,2003.0,0.0,135.0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1024948,2003-12-27,61131003.0,8.0,38.661210,-121.732690,2003,12,NaN,2003.0,NaN,NaN,1,6
1024949,2003-12-28,61131003.0,8.0,38.661210,-121.732690,2003,12,NaN,2003.0,NaN,NaN,2,6
1024950,2003-12-29,61131003.0,8.0,38.661210,-121.732690,2003,12,NaN,2003.0,NaN,NaN,3,6
1024951,2003-12-30,61131003.0,8.0,38.661210,-121.732690,2003,12,NaN,2003.0,NaN,NaN,4,6


In [ ]:
# Drop the multi-index and reset index
interpolated_data.reset_index(drop=True, inplace=True)

In [ ]:
interpolated_data.drop(columns=['count', 'month', 'day', 'aq_vuh'], inplace=True)
interpolated_data

,Date,Site.ID,Daily.Mean.PM2.5.Concentration,SITE_LATITUDE,SITE_LONGITUDE,year,school_year,consecutive_missing,max_consecutive_missing
0,2003-01-01,60010007.0,NaN,37.687526,-121.784217,2003,2002.0,1,2
1,2003-01-02,60010007.0,NaN,37.687526,-121.784217,2003,2002.0,2,2
2,2003-01-03,60010007.0,24.2,37.687526,-121.784217,2003,2002.0,0,2
3,2003-01-04,60010007.0,19.3,37.687526,-121.784217,2003,2002.0,1,2
4,2003-01-05,60010007.0,14.4,37.687526,-121.784217,2003,2002.0,2,2
...,...,...,...,...,...,...,...,...,...
1025537,2022-12-27,61131003.0,15.8,38.661210,-121.732690,2022,2022.0,2,5
1025538,2022-12-28,61131003.0,12.1,38.661210,-121.732690,2022,2022.0,3,5
1025539,2022-12-29,61131003.0,8.4,38.661210,-121.732690,2022,2022.0,4,5
1025540,2022-12-30,61131003.0,4.7,38.661210,-121.732690,2022,2022.0,5,5


In [ ]:
interpolated_data.groupby(['Site.ID', 'school_year']).ngroups

2950

In [ ]:
interpolated_data.to_csv("./interpolated_aq_180.csv")